# Dijkstra's Shortest Path Algorithm

This algorithm has a few characteristics

- Starts at a single node
- Returns the shortest path from that starting node to every other node
- Major Constraint: Graph must ONLY have **non-negative edges** with weights or values on each edge. If all edges were the same 1 step, we would use BFS instead!
  - Dijkstra's is essentially BFS with a priority queue/heap for weighted graphs
- Can be used on directed and undirected graphs
- Time complexity typically O(E* log(V))

## Implementation

In [55]:
import heapq

'''
Run Dijkstra's algorithm on an undirected graph to find the shortest path
from a starting node to an ending node. If there is no path between the
starting node and the destination node the returned value is set to be
float('inf').
'''
def dijkstra(graph, starting_node):
  # Maintain an array of the minimum distance to each node

  distances = {vertex: float('inf') for vertex in graph}
  prev = {vertex: None for vertex in graph} # Optional for building shortest path
  distances[starting_node] = 0

  pq = [(0, starting_node)]

  # Set used to track which nodes have already been visited.
  visited = set()
  while pq:
    curr_dist, curr_node = heapq.heappop(pq)

    if curr_node not in visited:
      visited.add(curr_node)
      # We already found a better path before we got to processing this node so we can ignore it.
      # Need to also do a check if curr node is not in graph in case of directed graphs
      if curr_dist > distances[curr_node] or curr_node not in graph:
        continue
      
      for neighbor, weight in graph[curr_node].items():
        # You cannot get a shorter path by revisiting
        # a node you have already visited before.
        if neighbor in visited:
          continue
        
        new_distance = curr_dist + weight
  
        if new_distance < distances[neighbor]:
          prev[neighbor] = curr_node # Optional: Useful for building paths later
          distances[neighbor] = new_distance
          heapq.heappush(pq, (new_distance, neighbor))
      
  return distances, prev

## Creating a Graph For Testing Dijkstra's

In [56]:
graph = {'Reykjavik': {'Oslo': 5, 'London': 4}, 
         'Oslo': {'Berlin': 1, 'Moscow': 3, 'Reykjavik': 5}, 
         'Moscow': {'Belgrade': 5, 'Athens': 4, 'Oslo': 3}, 
         'London': {'Reykjavik': 4}, 
         'Rome': {'Berlin': 2, 'Athens': 2}, 
         'Berlin': {'Oslo': 1, 'Rome': 2}, 
         'Belgrade': {'Moscow': 5, 'Athens': 1}, 
         'Athens': {'Belgrade': 1, 'Moscow': 4, 'Rome': 2}
        }

distances, prev_nodes = dijkstra(graph, 'Reykjavik')

print(distances)
print(prev_nodes)

{'Reykjavik': 0, 'Oslo': 5, 'Moscow': 8, 'London': 4, 'Rome': 8, 'Berlin': 6, 'Belgrade': 11, 'Athens': 10}
{'Reykjavik': None, 'Oslo': 'Reykjavik', 'Moscow': 'Oslo', 'London': 'Reykjavik', 'Rome': 'Berlin', 'Berlin': 'Oslo', 'Belgrade': 'Athens', 'Athens': 'Rome'}


## Building the Shortest Path with Dijkstra's

This fully relies on the `prev` array we populated in the dijkstra algorithm. If done correctly, getting the shortest of vertices from Node A to Node B is simple:

- Keep going down the previous nodes dictionary, starting with the target node
- Append each node to a list
- Once the node = the starting node, don't forget to append the start node as well.
- Return the path in reverse

In [57]:
def shortest_path(previous_nodes, shortest_path, start_node, target_node):
    path = []
    # We must start with the end since our previous_nodes dict will have vertices pointing to the previous node
    node = target_node
    
    while node != start_node:
        path.append(node)
        node = previous_nodes[node]
 
    # Add the start node manually
    path.append(start_node)

    return list(reversed(path))

path = shortest_path(prev_nodes, distances, 'Reykjavik', 'Belgrade')

print("Found the following best path with a value of {}.".format(distances['Belgrade']))
# Simply reverse the nodes and you have the path from start to end!
print(" -> ".join(path))

# Visual to show distances
path_lengths = [str(distances[x]) for x in path]
print(" -> ".join(path_lengths))

Found the following best path with a value of 11.
Reykjavik -> Oslo -> Berlin -> Rome -> Athens -> Belgrade
0 -> 5 -> 6 -> 8 -> 10 -> 11


## Directed Graph with Djikstra's

There is only a minor different between using dijkstra's with undirected graphs like above, and using directed graphs.

- When creating the `distances` dict, make sure an entry is made per node in the entire graph, not just the src nodes. This is implicitly the case with undirected graphs, but one must be careful to explicitly list out all nodes on the graph in the case of directed graphs. The [network delay problem](https://leetcode.com/problems/network-delay-time/) is a good example of this.

In [58]:
import heapq

'''
Run Dijkstra's algorithm on an undirected graph to find the shortest path
from a starting node to an ending node. If there is no path between the
starting node and the destination node the returned value is set to be
float('inf').
'''
def dijkstra(graph, starting_node, nodes):
  # Maintain an array of the minimum distance to each node

  # With a directed graph, we must make sure that each and every vertex (not just src but also dst nodes) are written in
  distances = {vertex: float('inf') for vertex in nodes}
  prev = {vertex: None for vertex in graph} # Optional for building shortest path
  distances[starting_node] = 0

  pq = [(0, starting_node)]

  # Set used to track which nodes have already been visited.
  visited = set()
  while pq:
    curr_dist, curr_node = heapq.heappop(pq)

    if curr_node not in visited:
      visited.add(curr_node)
      # We already found a better path before we got to
      # processing this node so we can ignore it.
      # Need to also do a check if curr node is not in graph in case of directed graphs
      if curr_dist > distances[curr_node] or curr_node not in graph:
        continue
      
      for neighbor, weight in graph[curr_node].items():
        # You cannot get a shorter path by revisiting
        # a node you have already visited before.
        if neighbor in visited:
          continue
        
        new_distance = curr_dist + weight
  
        if new_distance < distances[neighbor]:
          prev[neighbor] = curr_node # Optional: Useful for building path laterS
          distances[neighbor] = new_distance
          heapq.heappush(pq, (new_distance, neighbor))
      
  return distances, prev

## Analysis:

Time:

- Building distances dict takes O(V) since we add each vertex of the graph to it.
- while loop executes O(E) times based on how many times we add to the priority queue, which is the number of edges
- for loop executes O(E) times over edges and is within the while loop. the `if curr_dist > distances[curr_node]` ensures we only process each vertex once.
  - The use of a priority queue/min-heap also ensures we find the shortest path to vertices first
- Priority Queue operations to add and delete are log(E)

Therefore, runtime is O(V + ElogE)